In [1]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [2]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [4]:
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [5]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [6]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
# 跟line 溝通用
line_bot_api = LineBotApi("vtaRo4pkLqIZjk50gBJ53fFtMpg+8unl4TvLKbyvENmbWkzdriMNIeVm87ZWRBdTeDL7YM4hGte+BrNMmYscJ1WNPvCQbheXUbVevrlKB3zP3CaeRjbPQRPHMsqW+037bLOogqjOrh5ssRj53kqQMQdB04t89/1O/w1cDnyilFU=")
# 收發訊息用
handler = WebhookHandler("de385af73d2121e65ca6fc0490991b02")


In [7]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [12]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent, ImageMessage, TextSendMessage, VideoMessage, AudioMessage
)
# 用戶告訴handler 當收到圖片消息的時候執行下面
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    # 回應圖片上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    # 根據消息的id 向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    # 存在本地檔 以圖片方式儲存
    with open("/content/drive/MyDrive/line_bot_content/" + event.message.id + '.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [13]:
'''
影片消息，留給你
'''
# 用戶告訴handler 當收到影片消息的時候執行下面
@handler.add(MessageEvent, message=VideoMessage)
def handle_message(event):
    # 回應影片上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Video has Upload'+ ' ' + event.message.id))
    # 根據消息的id 向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    # 存在本地檔 以影片方式儲存
    with open("/content/drive/MyDrive/line_bot_content/" + event.message.id + '.mp4', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [14]:
'''
音訊消息，留給你
'''

# 用戶告訴handler 當收到音訊消息的時候執行下面
@handler.add(MessageEvent, message=AudioMessage)
def handle_message(event):
    # 回應音訊上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Sound has Upload'+ ' ' + event.message.id))
    # 根據消息的id 向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    # 存在本地檔 以音訊方式儲存
    with open("/content/drive/MyDrive/line_bot_content/" + event.message.id + '.aac', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [15]:
# 主程序運行
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2f70-35-185-92-95.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"video","id":"17811265767872","duration":14500,"contentProvider":{"type":"line"}},"webhookEventId":"01GVMJYG5WVPJHGNFR9GFNDBQ3","deliveryContext":{"isRedelivery":false},"timestamp":1678949433280,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"99bd7d8ff6eb4ddf827ba6b4058244a8","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:50:36] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"audio","id":"17811271734082","duration":1311,"contentProvider":{"type":"line"}},"webhookEventId":"01GVMK0SHZF2PQY6C2DMP6BYZ9","deliveryContext":{"isRedelivery":false},"timestamp":1678949508533,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"a673237343ee450e86f2fb000a6f1b2a","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:51:50] "POST / HTTP/1.1" 200 -
